<a href="https://colab.research.google.com/github/AnnaKodash/DS_learning/blob/main/Math%20/math14/Gibrid%2BDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808331 sha256=f94dc021091db894c566e309b2c4f45fd2a3bc37527dcf6d8039759a3d86d577
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [2]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

In [3]:
import pandas as pd

In [4]:
ratings = pd.read_csv('ratings.csv') # Поставленные оценки
books = pd.read_csv('books.csv') # Информация о книгах
tags = pd.read_csv('tags.csv') # Информация о тегах
book_tags = pd.read_csv('book_tags.csv') # Книги с тегами

In [5]:
books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [6]:
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [5]:
book_tags = pd.merge(book_tags, books[['book_id', 'goodreads_book_id']], how='left', on='goodreads_book_id')

In [6]:
book_tags[book_tags['goodreads_book_id']==5]

,goodreads_book_id,tag_id,count,book_id
300,5,11557,40087,18
301,5,11305,39330,18
302,5,8717,17944,18
303,5,33114,12856,18
304,5,30574,11909,18
...,...,...,...,...
395,5,20781,299,18
396,5,32345,298,18
397,5,12600,282,18
398,5,3379,277,18


In [12]:
tags

,tag_id,tag_name
0,509,19th-century
1,923,20th-century
2,941,21st-century
3,1499,abuse
4,1540,action
...,...,...
329,33114,young-adult
330,33121,young-adult-fantasy
331,33124,young-adult-fiction
332,33165,youth


In [7]:
book_tags = pd.merge(book_tags, tags[['tag_id']], how='inner', on='tag_id')

In [14]:
book_tags

,goodreads_book_id,tag_id,count,book_id
0,1,11305,37174,27
1,2,11305,3441,21
2,3,11305,47478,2
3,5,11305,39330,18
4,6,11305,38378,24
...,...,...,...,...
300733,16163690,17450,639,5021
300734,17257990,17450,576,7445
300735,17257993,17450,539,7434
300736,20603820,17450,233,9027


In [8]:
#!pip install scipy==1.10
from scipy.sparse import csr_matrix

In [9]:
ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id,ratings.book_id))) # Передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)

In [10]:
meta_matrix  = csr_matrix(([1]*len(book_tags),(book_tags.book_id,book_tags.tag_id)))

In [11]:
ratings_matrix.mean()

0.007086188900997592

In [12]:
model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

In [13]:
train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

In [14]:
model = model.fit(
    train, # Обучающая выборка
    item_features=meta_matrix, # Признаки товаров
    epochs=10, # Количество эпох
    verbose=True # Отображение обучения
)

Epoch: 100%|██████████| 10/10 [12:21<00:00, 74.14s/it]


In [16]:
recom = precision_at_k(model, test, item_features=meta_matrix)

In [18]:
recom.mean()

0.023226766

In [19]:
pip install tensorflow

In [20]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [21]:
df = pd.read_csv('ratings.csv')

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
train, test = train_test_split(
    df, # Общая выборка
    test_size=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

In [25]:
train

,book_id,user_id,rating
341848,3423,4608,2
964349,9811,36373,5
645459,6485,2957,4
74960,750,42400,3
358670,3591,36886,5
...,...,...,...
259178,2594,26266,4
365838,3663,27212,5
131932,1320,31839,4
671155,6746,34952,2


In [26]:
n_books = df["book_id"].nunique()
print(n_books)
n_users = df["user_id"].nunique()
print(n_users)

10000
53424


In [27]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

In [28]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [29]:
conc = Concatenate()([book_vec, user_vec])

In [30]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

In [31]:
model2 = Model([user_input, book_input], out)

In [32]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

In [33]:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 194s 8ms/step - loss: 0.8005
Epoch 2/5
24544/24544 [==============================] - 181s 7ms/step - loss: 0.6883
Epoch 3/5
24544/24544 [==============================] - 188s 8ms/step - loss: 0.6646
Epoch 4/5
24544/24544 [==============================] - 177s 7ms/step - loss: 0.6444
Epoch 5/5
24544/24544 [==============================] - 184s 7ms/step - loss: 0.6259


In [34]:
model2.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 [==============================] - 11s 2ms/step - loss: 0.7117


0.7116799354553223